## Package Import

In [1]:
import pandas as pd
import openpyxl
import os
import numpy as np

## Set Pandas Display Options

In [2]:
# enables showing both long and wide tables
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

## Create function to truncate comment field length

In [3]:
def auto_truncate(val):
    return val[:200]

## Data Inputs
For now, CARIBOU only in the input sheets. Codes have not been well defined for the other species. Further consultation needed for them.

In [4]:
# Paths to raw code tables, changing these tables will change how values are coded.
# codetable is an export of the BCTW code table as csv
codetable = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\incoming_template_tables\code_202204200956.csv"

# codeheader is an export of the BCTW code header table as csv
codeheader = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\incoming_template_tables\code_header_202204200958.csv" 

# codelookup is a custom table that maps Casylys metadata to BCTW db fields, code headers, and what type of field it is for updating.
codelookup = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\inputs\incoming_template_tables\codefieldlookup.csv"

In [5]:
# define incoming templates, require the device AND the animal
critter_in = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\test_data\BCTW_Telemetry_Template_20211209-R6.xlsx"
critter_in_sheet = "BCTW_Critter_Template"

device_in = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\test_data\BCTW_Device_Template_20220329R--R6.xlsx"
device_in_sheet = "BCTW_Device_Template"

In [6]:
# animal is a blank copy of the BCTW animal table
animal = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\outputs\BCTW_bulk_import_animal_template.csv"

# device is a blank copy of the BCTW device table
device = r"\\sfp.idir.bcgov\s140\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\bctw-migrate\templates\outputs\BCTW_bulk_import_device_template.csv"

## Data Output Location

In [7]:
outputloc = r"\\level\s40006\ESD\EI\Wildlife\WSI\Caribou_Program\Projects\BCTW\dataloading_scripting\loading_tables"

## Create Dataframes For Input Code Tables
These are used to classify data later

In [8]:
# dfcodetable and dfcodeheader filter out "deleted" data, will automatically drop inactive codes and headers, this logic delete on valid_to is used thruought BCTW
# indexes have been set on all the dataframes to enable seamless joining later on, do not change index fields
dfcodetable = pd.read_csv(codetable,index_col="code_header_id")
dfcodetable = dfcodetable[dfcodetable["valid_to"].isnull()]

dfcodeheader = pd.read_csv(codeheader,index_col="code_header_id")
dfcodeheader = dfcodeheader[dfcodeheader["valid_to"].isnull()]

dfcodelookup = pd.read_csv(codelookup,index_col='codeheader')

## Build Active Code Table
This table is built automatically by filtering out unused codes and code headers and then joining on the code map for which Casylys data goes to which db field.

In [9]:
activecode = dfcodetable.join(dfcodeheader,on="code_header_id", rsuffix="_header",sort=True)
activecode = activecode[["code_id","code_name","code_description","code_header_name"]]

In [10]:
finalcodetable = activecode.join(dfcodelookup)
finalcodetable = finalcodetable[finalcodetable["dbfield"].notnull()]

In [11]:
dfcodelookup= dfcodelookup[dfcodelookup.index.notnull()]

In [12]:
dfcodelookup

,Casylys,dbtable,dbfield,type
codeheader,,,,
1.0,Region,animal,region,code
139.0,Caribou Population Unit,animal,population_unit,code
181.0,Sex,animal,sex,code
179.0,Life Stage,animal,life_stage,code
177.0,Calf at Heel,animal,juvenile_at_heel,code
147.0,Collar Type,collar,device_type,code
4.0,Collar Make,collar,device_make,code
145.0,Satellite Network,collar,satellite_network,code
3.0,Animal Status,animal,animal_status,code


## Load Metadata sheet

In [13]:
# this fieldlist limits Casylys incoming data to the userful stuff
critter_fieldlist=['Region',
'Regional Contact',
'Regional Review',
'Regional Contact Comments',
'Project',
'Species',
'Caribou Ecotype',
'Caribou Population Unit',
'WLH ID',
'Animal ID',
'Sex',
'Life Stage',
'Calf at Heel',
'Ear Tag Right',
'Ear Tag Left',
'Device ID',
'Re-capture',
'Reg_Key',
'Trans-location',
'Collar Make',
'Capture Date',
'Capture Date Year',
'Capture Date Month',
'Capture UTM Zone',
'Capture UTM Easting',
'Capture UTM Northing',
'Capture Latitude',
'Capture Longitude',
'Release Date',
'Animal Status',
'Deployed',
'Collar Status Details',
'Deactivated?',
'Collar Retrieved?',
'Mortality Date',
'Malfunction Date',
'Malfunction Type',
'Retrieval Date',
'Mortality UTM Zone',
'Mortality UTM Easting',
'Mortality UTM Northing',
'Mortality Latitude',
'Mortality Longitude',
'Max Transmission Date',
'Previous Regional Comments',
'Mortality Review',
'Comments'
]

In [14]:
# device field list
device_fieldlist = ['device_make',
'device_model',
'device_id',
'device_deployment_status',
'device_status',
'device_condition',
'device_type',
'frequency',
'frequency_unit',
'satellite_network',
'first_activation_month',
'first_activation_year',
'fix_interval',
'fix_interval_rate',
'activation_comment',
'activation_status',
'camera_device_id',
'dropoff_device_id',
'dropoff_frequency',
'dropoff_frequency_unit',
'malfunction_comment',
'malfunction_date',
'device_malfunction_type',
'offline_date',
'offline_type',
'offline_comment',
'retrieval_date',
'retrieved',
'retrieval_comment',
'device_comment'
]

In [25]:
# pass incoming into a dataframe
dfcritter_in = pd.read_excel(critter_in,sheet_name=critter_in_sheet, engine='openpyxl',usecols=critter_fieldlist, converters = {'Comments': auto_truncate, 'Mortality Review': auto_truncate})
#dfdevice_in = pd.read_excel(device_in, sheet_name=device_in_sheet, engine='openpyxl',usecols=device_fieldlist, converters = {'Comments': auto_truncate})

C:\Users\Bauger\Anaconda3\envs\bctw\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## Cell below uses the the active codes to reclassify metadata tables, output is in the same format as input but with BCTW codes for applicable fields

In [22]:
# this code converts the BCTW description value from step above into the BCTW code value, output will be fields filled with integers
for i in dfcodelookup.index.values:
    x = dfcodelookup['Casylys'][int(i)]
    shortcodelist = finalcodetable.loc[int(i)]
    mapdict= dict(zip(shortcodelist.code_description, shortcodelist.code_id))
    if x in dfcritter_in.columns:
        dfcritter_in[x] = dfcritter_in[x].map(mapdict)
        print(("{} has been updated!").format(x))
    else:
        print(("{} not in table!").format(x))

Region has been updated!
Caribou Population Unit has been updated!
Sex has been updated!
Life Stage has been updated!
Calf at Heel has been updated!
Collar Type not in table!
Collar Make has been updated!
Satellite Network not in table!
Animal Status has been updated!
Deployed has been updated!
Collar Status Details has been updated!
Malfunction Type has been updated!


## Convert all boolean fields from y/n to true/false

In [ ]:
# loads codelookup into a new dataframe to identify the boolean fields
dfboollookupfields = pd.read_csv(codelookup,index_col="type")
dfboollookupfields = dfboollookupfields.loc['bool']

In [ ]:
# boolean field dictionary, null will be left as null
booldict = {'Y':'True','N':'False'}

In [ ]:
# apply the dictionary to the boolean fields, output should be either True or False, replaces all Y and N values
for f in dfboollookupfields['Casylys']:
    dfcritter_in[f] = dfcritter_in[f].map(booldict)
    

In [ ]:
dfcritter_in

## Load BCTW output templates and prepare to have data migrated to them

### Change metadata field names to bctw

In [ ]:
# load codelookup again to build a dictionary of Casylys to BCTW field names
dfcodelookupfields = pd.read_csv(codelookup)
dfcodelookupfields = dfcodelookupfields[dfcodelookupfields['dbfield'].notnull()]

In [ ]:
# builds field map dictionary
fielddict = dict(zip(dfcodelookupfields.Casylys,dfcodelookupfields.dbfield))

In [ ]:
# renames all fields in the above dictionary
dfmetadata=dfmetadata.rename(columns=(fielddict))

In [ ]:
dfmetadata

## Remove spaces and special characters from ear tag ID fields to get them under 20 characters

In [ ]:
dfmetadata['ear_tag_right_id'] = dfmetadata['ear_tag_right_id'].str.replace('\W', '')
dfmetadata['ear_tag_left_id'] = dfmetadata['ear_tag_left_id'].str.replace('\W', '')

## Sort out logic for offline data, if mort data or malfunciton date is filled in, offline_date equals one, if both null, offline_date is null

In [ ]:
# Create a concatenated column for malfunction and mortalilty date columns
# add retrieval date to date list
# Written by Orla O!
dfmetadata['mort_mal'] = dfmetadata[['malftunction_date', 'mortality_date','retrieval_date']].apply(lambda x: ''.join(str(value) for value in x), axis=1)
dfmetadata['mort_mal'] = dfmetadata['mort_mal'].replace("NaTNaTNaT", 0, regex=True)

# replace 'offline_date' with a null if 'mort_mal' is 0 (i.e. null)
dfmetadata['offline_date'] = np.where(dfmetadata['mort_mal'] == 0, np.nan, dfmetadata['offline_date'])

# delete 'mort_mal' column
dfmetadata = dfmetadata.drop(columns=['mort_mal'])

In [ ]:
dfmetadata

## Create critter code field to identify duplicate critters, this will capture recaptured data or duplicate wlh_id
Zero out wlh_id that are blank, so value can still be unique

In [ ]:
# fill blank wlh_id with 0 so it will still create a code 
dfmetadata['wlh_id']= dfmetadata['wlh_id'].fillna(0)
# create crittercode field to determine if animal has more than 1 record
dfmetadata['crittercode']= dfmetadata['wlh_id'].astype(str).str.cat(dfmetadata['animal_id'].astype(str), sep="_")

In [ ]:
# append duplicate field information to the dataframe
dfmetadata['duplicate']=dfmetadata.duplicated(subset='crittercode',keep=False)

## Load blank csv templates and put matching fields into them and export as csv
Animal output split into recapture and not recpature. Recaptures needs to be handled differently and require the animal uid to be assigned to it. Might have to consider manually adding those records. Solution unclear.

In [ ]:
# blank csv used to format the output of the dataframes, this template was downloaded from BCTW data import wizard
dfanimal = pd.read_csv(animal)
dfdevice = pd.read_csv(device)

In [ ]:
# filter dfmetadata for duplicates
dfmetadata['created_by_user_id']=0
dfmetadatadupe = dfmetadata[dfmetadata['duplicate']==True]
dfmetadatanotdupe = dfmetadata[dfmetadata['duplicate']==False]


In [ ]:
# load blank templates into df
dfdeviceout = pd.concat([dfdevice, dfmetadatanotdupe],join='inner')
dfdeviceout['valid_to']=dfdeviceout['offline_date']
dfdeviceoutrecap = pd.concat([dfdevice, dfmetadatadupe],join='inner')
dfdeviceoutrecap['valid_to']=dfdeviceout['offline_date']
dfanimalout = pd.concat([dfanimal, dfmetadatanotdupe],join='inner')
dfanimaloutrecap = pd.concat([dfanimal, dfmetadatadupe],join='inner')

In [ ]:
# concatenate animal fields into template, only fields that exist in the output will be copied
outpatha = metadatasheet +'_'+ 'dfanimal.csv'
a_output = os.path.join(outputloc,outpatha)
dfanimalout.to_csv(a_output,index=False)

In [ ]:
# concatenate animal fields into template, export only recapture records
outpathar = metadatasheet +'_'+ 'recapture_dfanimal.csv'
ar_output = os.path.join(outputloc,outpathar)
dfanimaloutrecap.to_csv(ar_output,index=False)

In [ ]:
# concatenate device fields into template, only fields that exist in the output will be copied
outpathd = metadatasheet +'_'+ 'dfdevice.csv'
d_output = os.path.join(outputloc, outpathd)
dfdeviceout.to_csv(d_output,index=False)

In [ ]:
# concatenate device fields into template, only fields that exist in the output will be copied
outpathdd = metadatasheet +'_'+ 'duplicate_dfdevice.csv'
dd_output = os.path.join(outputloc, outpathdd)
dfdeviceoutrecap.to_csv(dd_output,index=False)